# Cleaning Election Data
Code from https://pointsunknown.nyc/web%20mapping/mapbox/python/pandas/geopandas/2021/07/14/08A_WebmappingElectionData.html

## Importing libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install geopandas
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# !pip freeze
# saves your installed packages to your google drive

## Import base data

# Load data

Download and upload these to Colab

US Counties - from [U.S. Census Bureau - Cartographic Boundary Files.](https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_5m.zip)

US States - from [U.S. Census Bureau - Cartographic Boundary Files](https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_state_5m.zip)

US Presidential Election Results at the County Level - from [MIT Election Data + Science Lab](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ)

In [4]:
# Load election data
electionData = pd.read_csv('countypres_2000-2020.csv')

In [5]:
# Filter to just 2020

# In Python, every time you get a value is trying to be set on a copy of a slice from a DataFrame warning, 
# it's because changes made to the new election2020 will have nothing to do with changes to the original electionData

# The way to remove this warning is to make it a deep copy, which tells pandas specifically to make it an independent copy

election2020 = electionData[electionData.year == 2020].copy(deep=True)

### Create states data

In [6]:
# Create statesData frame grouping votes from 2020 for each candidate
statesData = pd.pivot_table(election2020, index='state', values='candidatevotes', columns='candidate', aggfunc=np.sum).reset_index()

# Rename the index so it doesn't say "candidate"
statesData = statesData.rename_axis(None, axis=1)

statesData.head()

,state,DONALD J TRUMP,JO JORGENSEN,JOSEPH R BIDEN JR,OTHER
0,ALABAMA,1441170.0,NaN,849624.0,32488.0
1,ALASKA,189951.0,8897.0,153778.0,4943.0
2,ARIZONA,1661686.0,51465.0,1672143.0,0.0
3,ARKANSAS,760647.0,13133.0,423932.0,21357.0
4,CALIFORNIA,6006429.0,187895.0,11110250.0,196307.0


In [7]:
# Fill NAs
statesData.fillna(0, inplace=True)

In [8]:
# Calculate total number of votes
statesData['Total'] = statesData['DONALD J TRUMP'] + statesData['JOSEPH R BIDEN JR'] + statesData['JO JORGENSEN'] + statesData['OTHER']

# Calculate winners - idmax means look at the maximum value
statesData['Winner'] = statesData[['DONALD J TRUMP','JOSEPH R BIDEN JR','JO JORGENSEN','OTHER']].idxmax(axis=1)
# Convert Winner column to title case
statesData['Winner'] = statesData['Winner'].str.title()

# Calculate percent of votes the winner got
# numpy where - where the winner is Donald J Trump, apply the value Trump's votes / total votes and where it's NOT that, use Biden's votes / total votes
statesData['WnrPerc'] = np.where(statesData['Winner']=='Donald J Trump', statesData['DONALD J TRUMP']/statesData['Total'], statesData['JOSEPH R BIDEN JR']/statesData['Total'])

# A better alternative for this might be and then you could then do this for everyone else
# for index, row in statesData.iterrows():
#   if row['Winner'] == 'Donald J Trump':
#     statesData.iloc[index]['WnrPerc'] = row['DONALD J TRUMP']/row['Total']

In [9]:
statesData.head()

,state,DONALD J TRUMP,JO JORGENSEN,JOSEPH R BIDEN JR,OTHER,Total,Winner,WnrPerc
0,ALABAMA,1441170.0,0.0,849624.0,32488.0,2323282.0,Donald J Trump,0.620316
1,ALASKA,189951.0,8897.0,153778.0,4943.0,357569.0,Donald J Trump,0.531229
2,ARIZONA,1661686.0,51465.0,1672143.0,0.0,3385294.0,Joseph R Biden Jr,0.493943
3,ARKANSAS,760647.0,13133.0,423932.0,21357.0,1219069.0,Donald J Trump,0.623957
4,CALIFORNIA,6006429.0,187895.0,11110250.0,196307.0,17500881.0,Joseph R Biden Jr,0.634839


In [10]:
# Drop if there's an NA in county_fips because then we can't map it
election2020.dropna(subset=['county_fips'], inplace=True)

In [11]:
# Convert types to keep the numbers
# zfill means fill with 0s until there are 5 digits in the number
election2020['county_fips'] = election2020['county_fips'].astype('int').astype('str').str.zfill(5)

In [12]:
# Rename county fips to GEOID because our shapefile will say GEOID
election2020.rename(columns={'county_fips':'GEOID'}, inplace=True)

### Create counties data, same way we did state sabove

In [15]:
# Pivot data
countiesData = pd.pivot_table(election2020, index='GEOID', values='candidatevotes', columns='candidate', aggfunc=np.sum).reset_index()

# Remove index name
countiesData = countiesData.rename_axis(None, axis=1)

countiesData.head()

,GEOID,DONALD J TRUMP,JO JORGENSEN,JOSEPH R BIDEN JR,OTHER
0,01001,19838.0,NaN,7503.0,429.0
1,01003,83544.0,NaN,24578.0,1557.0
2,01005,5622.0,NaN,4816.0,80.0
3,01007,7525.0,NaN,1986.0,84.0
4,01009,24711.0,NaN,2640.0,237.0


In [16]:
# Fill NA
countiesData.fillna(0, inplace=True)

In [17]:
# Create variables
countiesData['Total'] = countiesData['DONALD J TRUMP'] + countiesData['JOSEPH R BIDEN JR'] + countiesData['JO JORGENSEN'] + countiesData['OTHER']
countiesData['Winner'] = countiesData[['DONALD J TRUMP','JOSEPH R BIDEN JR','JO JORGENSEN','OTHER']].idxmax(axis=1)
countiesData['Winner'] = countiesData['Winner'].str.title()
countiesData['WnrPerc'] = np.where(countiesData['Winner']=='Donald J Trump', countiesData['DONALD J TRUMP']/countiesData['Total'], countiesData['JOSEPH R BIDEN JR']/countiesData['Total'])

## Load shapefiles

In [18]:
# Could use the links or the files we downloaded
# counties = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_5m.zip')
# states = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_state_5m.zip')

# 5m (5 meters) shows the level of detail that we have. 5m is highest resolution, we could use a lower one though because we won't be zooming in that much
counties = gpd.read_file('cb_2020_us_county_5m.zip')
states = gpd.read_file('cb_2020_us_state_5m.zip')

## Merge election data with geographies

In [19]:
# Merging the states datasets
statesData['state'] = statesData['state'].str.title()
states['NAME'] = states['NAME'].str.title()
statesData.rename(columns={'state':'NAME'}, inplace=True)
statesElections = states.merge(statesData, on='NAME')
statesElections = statesElections[['NAME','geometry','DONALD J TRUMP','JOSEPH R BIDEN JR','JO JORGENSEN','OTHER','Total','Winner','WnrPerc']].copy(deep=True)
statesElections.rename(columns={'NAME':'State','DONALD J TRUMP':'Trump','JOSEPH R BIDEN JR':'Biden','JO JORGENSEN':'Jorgensen','OTHER':'Other'}, inplace=True)

In [20]:
# Merging the counties datasets
countiesElections = counties.merge(countiesData, on='GEOID')
countiesElections = countiesElections[['NAME','STATE_NAME','geometry','DONALD J TRUMP','JOSEPH R BIDEN JR','JO JORGENSEN','OTHER','Total','Winner','WnrPerc']].copy(deep=True)
countiesElections.rename(columns={'NAME':'County','STATE_NAME':'State','DONALD J TRUMP':'Trump','JOSEPH R BIDEN JR':'Biden','JO JORGENSEN':'Jorgensen','OTHER':'Other'}, inplace=True)

In [21]:
# Creating the county points dataset - this creates the center point for each geography
countiesPoints = countiesElections.copy(deep=True)
countiesPoints['point'] = countiesPoints.representative_point()
countiesPoints['geometry'] = countiesPoints['point']
countiesPoints.drop(columns=['point'], inplace=True)

## Export and save geojsons

In [22]:
# Exporting datasets as GeoJSONs
countiesElections.to_file('countiesElections.geojson', driver='GeoJSON')
statesElections.to_file('statesElections.geojson', driver='GeoJSON')
countiesPoints.to_file('countiesPoints.geojson', driver='GeoJSON')

In [23]:
# Download files (you can also just download it from the sidebar)
from google.colab import files
files.download('countiesElections.geojson')
files.download('statesElections.geojson') 
files.download('countiesPoints.geojson') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>